# Olist E-commerce Data Quality Analysis

This notebook demonstrates the key data quality issues found in the Olist dataset that require special handling in our database loader.

In [1]:
from pathlib import Path

import pandas as pd

# Load the datasets
data_dir = Path("../data")
reviews_df = pd.read_csv(data_dir / "olist-ecommerce" / "olist_order_reviews_dataset.csv")
closed_deals_df = pd.read_csv(data_dir / "olist-marketing-funnel" / "olist_closed_deals_dataset.csv")
sellers_df = pd.read_csv(data_dir / "olist-ecommerce" / "olist_sellers_dataset.csv")

print("Loaded datasets:")
print(f"  - Order Reviews: {len(reviews_df):,} rows")
print(f"  - Closed Deals: {len(closed_deals_df):,} rows") 
print(f"  - Sellers: {len(sellers_df):,} rows")


Loaded datasets:
  - Order Reviews: 100,000 rows
  - Closed Deals: 842 rows
  - Sellers: 3,095 rows


## Issue 1: Duplicate Review IDs

The `order_reviews` dataset has duplicate `review_id` values - the same review ID appears for different orders.

In [2]:
# Check for duplicate review_ids
total_reviews = len(reviews_df)
unique_review_ids = reviews_df["review_id"].nunique()
duplicate_count = total_reviews - unique_review_ids

print("Review ID Analysis:")
print(f"  - Total reviews: {total_reviews:,}")
print(f"  - Unique review IDs: {unique_review_ids:,}")
print(f"  - Duplicate review IDs: {duplicate_count:,}")
print(f"  - Duplicate percentage: {duplicate_count/total_reviews*100:.1f}%")

# Show examples of duplicate review_ids
duplicates = (
    reviews_df[reviews_df.duplicated("review_id", keep=False)]
    .sort_values("review_id")
    .head(10)
)
print("\nFirst 10 duplicate review_ids:")
display(duplicates[["review_id", "order_id", "review_score"]].head(10))


Review ID Analysis:
  - Total reviews: 100,000
  - Unique review IDs: 99,173
  - Duplicate review IDs: 827
  - Duplicate percentage: 0.8%

First 10 duplicate review_ids:


,review_id,order_id,review_score
47045,00130cbe1f9d422698c812ed8ded1919,dfcdfc43867d1c1381bfaf62d6b9c195,1
30062,00130cbe1f9d422698c812ed8ded1919,04a28263e085d399c97ae49e0b477efa,1
63695,0115633a9c298b6a98bcbe4eee75345f,0c9850b2c179c1ef60d2855e2751d1fa,5
91399,0115633a9c298b6a98bcbe4eee75345f,78a4201f58af3463bdab842eea4bc801,5
57727,0174caf0ee5964646040cd94e15ac95e,74db91e33b4e1fd865356c89a61abf1f,1
93607,0174caf0ee5964646040cd94e15ac95e,f93a732712407c02dce5dd5088d0f47b,1
55259,017808d29fd1f942d97e50184dfb4c13,8daaa9e99d60fbba579cc1c3e3bfae01,5
99942,017808d29fd1f942d97e50184dfb4c13,b1461c8882153b5fe68307c46a506e39,5
20771,0254bd905dc677a6078990aad3331a36,5bf226cf882c5bf4247f89a97c86f273,1
96833,0254bd905dc677a6078990aad3331a36,331b367bdd766f3d1cf518777317b5d9,1


## Issue 2: Invalid Seller References

The `closed_deals` dataset contains `seller_id` values that don't exist in the `sellers` table, violating foreign key constraints.

In [3]:
# Check for invalid seller_id references
valid_seller_ids = set(sellers_df["seller_id"])
deals_with_seller_id = closed_deals_df.dropna(subset=["seller_id"])

invalid_sellers = deals_with_seller_id[
    ~deals_with_seller_id["seller_id"].isin(valid_seller_ids)
]

print("Seller ID Analysis:")
print(f"  - Total closed deals: {len(closed_deals_df):,}")
print(f"  - Deals with seller_id: {len(deals_with_seller_id):,}")
print(f"  - Valid seller_ids: {len(sellers_df):,}")
print(f"  - Invalid seller references: {len(invalid_sellers):,}")
print(f"  - Invalid percentage: {
    len(invalid_sellers)/len(deals_with_seller_id)*100:.1f
}%")

# Show examples of invalid seller_ids
print("\nFirst 5 invalid seller_ids:")
display(invalid_sellers[["mql_id", "seller_id", "business_segment"]].head())

Seller ID Analysis:
  - Total closed deals: 842
  - Deals with seller_id: 842
  - Valid seller_ids: 3,095
  - Invalid seller references: 462
  - Invalid percentage: 54.9%

First 5 invalid seller_ids:


,mql_id,seller_id,business_segment
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,car_accessories
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,food_drink
6,c3e30ed7ac989117c7e1e719b4ac128f,92d7568ad0c5c76fd7d341b2d46f24d6,computers
8,a90a37898cc5f2718385a2fb981caaff,0b28859cd04d23edefee9c591fb03cd8,household_utilities
10,4e4e70d504b4c0006c8287dedc99d0fc,f7a0d94e966c5665355a182d5b199fcf,construction_tools_house_garden


## Data Quality Handling

Our database loader handles these issues automatically:

1. **Duplicate Review IDs**: Uses `DISTINCT ON (review_id)` to keep only the first occurrence of each review_id, ensuring PRIMARY KEY constraint compliance.

2. **Invalid Seller References**: Filters out closed_deals with seller_ids that don't exist in the sellers table, ensuring FOREIGN KEY constraint compliance.

These fixes ensure data integrity while preserving the maximum amount of usable data for analysis.